<a href="https://colab.research.google.com/github/ParkSoobin1219/cs231n_assignments/blob/main/CIFAR10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Common**

In [1]:
#coding: utf-8
import torch
import torchvision 
from torchvision import transforms

In [2]:
#디바이스 지정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
#데이터셋 만들기
train_dataset = torchvision.datasets.CIFAR10('./', download=True)
test_dataset = torchvision.datasets.CIFAR10('./', train=False)
print(train_dataset)
print(test_dataset)

Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./
    Split: Train
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./
    Split: Test


In [ ]:
x_train_orig = train_dataset.data
y_train_orig = train_dataset.targets
x_test_orig = test_dataset.data
y_test_orig = test_dataset.targets

In [ ]:
# normalization!!
x_train = (x_train_orig[10000:] / 255.)
y_train = y_train_orig[10000:]
x_valid = (x_train_orig[:10000] / 255.)
y_valid = y_train_orig[:10000]

x_test = x_test_orig / 255.
y_test = y_test_orig


print('x_train shape : ', x_train.shape)
print('y_train shape : ', len(y_train))
print('x_valid shape : ', x_valid.shape)
print('y_valid shape : ', len(y_valid))
print('x_test shape : ', x_test.shape)
print('y_test shape : ', len(y_test))


#------------list(y)->torch.tensor, ndarray(x)->torch.tensor 변환------------
x_train = torch.tensor(x_train, dtype=torch.float32).to(device=device)
y_train = torch.tensor(y_train).to(device=device)
x_valid = torch.tensor(x_valid, dtype=torch.float32).to(device=device)
y_valid = torch.tensor(y_valid).to(device=device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device=device)
y_test = torch.tensor(y_test).to(device=device)

print('------------just for checking---------')
print(type(x_train))
print(type(y_train))
print(type(x_valid))
print(type(y_valid))
print(type(y_test))
print('----------------------------------------')

#============================================================================

print('x_train shape : ', x_train.shape)
print('y_train shape : ', y_train.shape)
print('x_valid shape : ', x_valid.shape)
print('y_valid shape : ', y_valid.shape)
print('x_test  shape : ', x_test.shape)
print('y_test  shape : ', y_test.shape)

x_train shape :  (40000, 32, 32, 3)
y_train shape :  40000
x_valid shape :  (10000, 32, 32, 3)
y_valid shape :  10000
x_test shape :  (10000, 32, 32, 3)
y_test shape :  10000
------------just for checking---------
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
----------------------------------------
x_train shape :  torch.Size([40000, 32, 32, 3])
y_train shape :  torch.Size([40000])
x_valid shape :  torch.Size([10000, 32, 32, 3])
y_valid shape :  torch.Size([10000])
x_test  shape :  torch.Size([10000, 32, 32, 3])
y_test  shape :  torch.Size([10000])


In [ ]:
trainingset = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(trainingset, batch_size=32, shuffle=True)
print(type(trainingset))
print(type(train_loader))

<class 'torch.utils.data.dataset.TensorDataset'>
<class 'torch.utils.data.dataloader.DataLoader'>


# **MLP**

In [ ]:
# MLP Model

class MLPNet(torch.nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        # self.fc1 = torch.nn.Linear(3072, 256)
        # self.fc2 = torch.nn.Linear(256, 128)
        # self.fc3 = torch.nn.Linear(128, 128)
        # self.fc4 = torch.nn.Linear(128, 10)

        self.fc1 = torch.nn.Linear(3072, 1024)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.fc3 = torch.nn.Linear(512, 256)
        self.fc4 = torch.nn.Linear(256, 10)


    def forward(self, x):
        x = x.reshape(-1, 3072)     #.reshape 이나 .view나 똑같은데 각각 먹히는 케이스가 따로 있으니까 걍 디버깅 해준대로 쓰셈
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = torch.nn.functional.relu(self.fc3(x))
        x = self.fc4(x)
        return x


mlpnet = MLPNet()
mlpnet = mlpnet.to(device=device)
print(mlpnet)

MLPNet(
  (fc1): Linear(in_features=3072, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=10, bias=True)
)


In [ ]:
# Optimization
criterion = torch.nn.CrossEntropyLoss() # CrossEntropyLoss function 사용
optimizer = torch.optim.Adam(mlpnet.parameters(), lr=0.001) #Adam optimization 사용

In [ ]:
# Train the model
import time

for epoch in range(10):
    start = time.time()
    total_loss = 0
    
 
    for xb , yb in train_loader:

        pred = mlpnet(xb)
        loss = criterion(pred, yb)
 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
 
        total_loss += loss.item()    

    with torch.no_grad():
        pred = mlpnet(x_train)
        acc = pred.data.max(1)[1].eq(y_train.data).sum()/len(x_train) * 100
        loss = criterion(pred, y_train)
    print(f"{time.time() - start} sec - loss : {total_loss/len(train_loader)} / acc : {acc}")

42.47835564613342 sec - loss : 1.89895291595459 / acc : 36.459999084472656
42.2156023979187 sec - loss : 1.7415872769355774 / acc : 39.72999954223633
43.15572738647461 sec - loss : 1.656908612537384 / acc : 42.162498474121094
42.97228741645813 sec - loss : 1.5914079795837401 / acc : 44.75749969482422
42.83901619911194 sec - loss : 1.547443834877014 / acc : 46.337501525878906
42.5416259765625 sec - loss : 1.5035449610233307 / acc : 44.55249786376953
42.39805889129639 sec - loss : 1.472073563814163 / acc : 49.23749923706055
42.42520809173584 sec - loss : 1.4446670634269714 / acc : 49.29249954223633
41.945802211761475 sec - loss : 1.4145916658878326 / acc : 50.47500228881836
42.619500398635864 sec - loss : 1.3922191262722015 / acc : 50.650001525878906


In [ ]:
# Test the training model
pred = mlpnet(x_test.to(device=device))
acc = pred.data.max(1)[1].eq(y_test.data).sum()/len(x_test) *100
print(f"acc : {acc}")

acc : 46.52000045776367


# **CNN**

In [ ]:
# CNN model

class CNN_Model(torch.nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, 5, 1, padding=2) #(input channel, output channel, kernel_size, stride, padding)
        self.conv2 = torch.nn.Conv2d(32, 16, 3, 1, padding=1) #input size = conv1-output channel
        self.fc1 = torch.nn.Linear(1024, 10)
    
    def forward(self, x):
        x = x.reshape(-1, 3, 32, 32) #(no.of batch, channel, height, width)
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.max_pool2d(x, 2)
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(x, 2)
        x = torch.nn.Flatten()(x)
        x = self.fc1(x)
        return x
 
cnn = CNN_Model()
cnn = cnn.to(device=device)

1. x -> [3x32x32]-------기준!!!!!!!!
2. self.conv1(x) -> [32x32x32]
3. relu(self.conv1(x)) -> [32x32x32]
4. max_pool2d(x,2) -> [16x16x32]
5. self.conv2(x) -> [16x16x16]
6. relu(self.conv2(x)) -> [16x16x16]
7. max_pool2d(x, 2) -> [8x8x16]
8. Flatten()(x) -> 1024


In [ ]:
# Optimization
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

In [ ]:
# Train the model
import time

for epoch in range(3):
    start = time.time()
    total_loss = 0
 
    for xb, yb in train_loader:
        
        
        pred = cnn(xb)
        loss = criterion(pred, yb)
 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
 
        total_loss += loss.item()
    
    with torch.no_grad():
        pred = cnn(x_train)
        acc = pred.data.max(1)[1].eq(y_train.data).sum()/len(x_train) * 100
        loss = criterion(pred, y_train)
    print(f"{time.time() - start} sec - loss : {loss} / acc : {acc}")

In [ ]:
# Test the CNN model

pred = cnn(x_test.to(device=device))
acc = pred.data.max(1)[1].eq(y_test.to(device=device).data).sum()/len(x_test) * 100
loss = criterion(pred, y_test.to(device=device))
print(f"loss : {loss} / acc : {acc}")